In [7]:
import kagglehub
import os
import shutil

# Check if the dataset already exists and download it if not found
if not os.path.exists('./Dataset'):
    os.mkdir('./Dataset')
    dataset_path = kagglehub.dataset_download("niharika41298/yoga-poses-dataset")
    shutil.move(str(dataset_path), './Dataset')
    print("Path to dataset files:", dataset_path)

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import mediapipe as mp

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)


def calculate_angle(a, b, c):
    """Calculate the angle between three points"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine_angle, -1, 1)))


def extract_features(image_path):
    """Extract both image and biomechanical features from a single image"""
    img = cv2.imread(image_path)
    if img is None:
        return None

    # Process image with MediaPipe
    results = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    if not results.pose_landmarks:
        return None

    # Get landmarks
    landmarks = results.pose_landmarks.landmark

    # Calculate key angles
    points = {
        'left_shoulder': [landmarks[11].x, landmarks[11].y],
        'right_shoulder': [landmarks[12].x, landmarks[12].y],
        'left_elbow': [landmarks[13].x, landmarks[13].y],
        'right_elbow': [landmarks[14].x, landmarks[14].y],
        'left_wrist': [landmarks[15].x, landmarks[15].y],
        'right_wrist': [landmarks[16].x, landmarks[16].y],
        'left_hip': [landmarks[23].x, landmarks[23].y],
        'right_hip': [landmarks[24].x, landmarks[24].y],
        'left_knee': [landmarks[25].x, landmarks[25].y],
        'right_knee': [landmarks[26].x, landmarks[26].y],
        'left_ankle': [landmarks[27].x, landmarks[27].y],
        'right_ankle': [landmarks[28].x, landmarks[28].y]
    }

    angles = {
        'left_elbow': calculate_angle(points['left_shoulder'], points['left_elbow'], points['left_wrist']),
        'right_elbow': calculate_angle(points['right_shoulder'], points['right_elbow'], points['right_wrist']),
        'left_shoulder': calculate_angle(points['left_elbow'], points['left_shoulder'], points['left_hip']),
        'right_shoulder': calculate_angle(points['right_elbow'], points['right_shoulder'], points['right_hip']),
        'left_hip': calculate_angle(points['left_shoulder'], points['left_hip'], points['left_knee']),
        'right_hip': calculate_angle(points['right_shoulder'], points['right_hip'], points['right_knee']),
        'left_knee': calculate_angle(points['left_hip'], points['left_knee'], points['left_ankle']),
        'right_knee': calculate_angle(points['right_hip'], points['right_knee'], points['right_ankle']),
        'spine': calculate_angle(points['left_shoulder'], points['left_hip'], points['left_ankle'])
    }

    # Prepare image for CNN
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0

    return {
        'image': img_normalized,
        'angles': list(angles.values()),
        'landmarks': points
    }


def process_dataset(dataset_path):
    """Process entire dataset and save features"""
    data = []
    pose_classes = [d for d in os.listdir(dataset_path)
                    if os.path.isdir(os.path.join(dataset_path, d))]

    for pose_class in pose_classes:
        class_dir = os.path.join(dataset_path, pose_class)
        image_files = [f for f in os.listdir(class_dir)
                       if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        for img_file in tqdm(image_files, desc=f"Processing {pose_class}"):
            img_path = os.path.join(class_dir, img_file)
            features = extract_features(img_path)

            if features is not None:
                features['class'] = pose_class
                data.append(features)

    # Convert to DataFrame
    df = pd.DataFrame(data)

    # Save processed data
    df.to_pickle('./data/yoga_dataset_processed.pkl')

    # Calculate pose standards
    pose_standards = df.groupby('class')['angles'].apply(
        lambda x: {
            'median': np.median(np.vstack(x), axis=0),
            'std': np.std(np.vstack(x), axis=0)
        }
    ).to_dict()

    with open('pose_standards.pkl', 'wb') as f:
        pickle.dump(pose_standards, f)

    return df, pose_standards


if __name__ == "__main__":
    dataset_path = r"Dataset\1\DATASET\TRAIN"
    df, standards = process_dataset(dataset_path)

Processing warrior2: 100%|██████████| 252/252 [00:33<00:00,  7.51it/s]


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle


def create_hybrid_model(num_classes=5):
    """Create a model that combines CNN and biomechanical features"""
    # Image branch (CNN)
    image_input = layers.Input(shape=(224, 224, 3), name='image_input')

    x = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)

    # Angle branch
    angle_input = layers.Input(shape=(9,), name='angle_input')
    a = layers.Dense(32, activation='relu')(angle_input)

    # Combined features
    combined = layers.concatenate([x, a])

    # Classifier
    z = layers.Dense(128, activation='relu')(combined)
    z = layers.Dropout(0.5)(z)
    output = layers.Dense(num_classes, activation='softmax')(z)

    return models.Model(inputs=[image_input, angle_input], outputs=output)


def prepare_data(df):
    """Prepare data for training"""
    # Convert images to array
    X_images = np.array([x for x in df['image']])

    # Convert angles to array
    X_angles = np.array([x for x in df['angles']])

    # Convert labels to one-hot
    class_to_idx = {cls: i for i, cls in enumerate(df['class'].unique())}
    y = tf.keras.utils.to_categorical(df['class'].map(class_to_idx))

    return X_images, X_angles, y, class_to_idx


def train_model():
    # Load processed data
    df = pd.read_pickle('yoga_dataset_processed.pkl')

    # Prepare data
    X_img, X_ang, y, class_to_idx = prepare_data(df)

    # Split data - USE THE SAME VARIABLE NAMES RETURNED FROM prepare_data()
    (X_img_train, X_img_val,
     X_ang_train, X_ang_val,
     y_train, y_val) = train_test_split(X_img, X_ang, y, test_size=0.2)

    # Create model
    model = create_hybrid_model(num_classes=len(class_to_idx))

    # Compile
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Define callbacks
    training_callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint('best_model.h5', save_best_only=True),
        ReduceLROnPlateau(factor=0.1, patience=3)
    ]

    # Train
    history = model.fit(
        x={'image_input': X_img_train, 'angle_input': X_ang_train},
        y=y_train,
        validation_data=({'image_input': X_img_val, 'angle_input': X_ang_val}, y_val),
        epochs=10,
        batch_size=32,
        callbacks=training_callbacks
    )

    # Save class mapping
    idx_to_class = {v: k for k, v in class_to_idx.items()}  # Reverse mapping
    with open('class_mapping.pkl', 'wb') as f:
        pickle.dump(idx_to_class, f)  # New: {0:'downdog', 1:'warrior'}

    return model, history


if __name__ == "__main__":
    model, history = train_model()

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 830ms/step - accuracy: 0.2770 - loss: 25.2309

26/26 ━━━━━━━━━━━━━━━━━━━━ 36s 949ms/step - accuracy: 0.2778 - loss: 25.0568 - val_accuracy: 0.4689 - val_loss: 4.6225 - learning_rate: 0.0010
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4514 - loss: 6.9466

26/26 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.4521 - loss: 6.8696 - val_accuracy: 0.6268 - val_loss: 1.2446 - learning_rate: 0.0010
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 801ms/step - accuracy: 0.4914 - loss: 1.4125

26/26 ━━━━━━━━━━━━━━━━━━━━ 22s 858ms/step - accuracy: 0.4906 - loss: 1.4124 - val_accuracy: 0.6029 - val_loss: 1.1218 - learning_rate: 0.0010
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 766ms/step - accuracy: 0.5315 - loss: 1.1589

26/26 ━━━━━━━━━━━━━━━━━━━━ 22s 829ms/step - accuracy: 0.5316 - loss: 1.1592 - val_accuracy: 0.6411 - val_loss: 1.0114 - learning_rate: 0.0010
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 21s 789ms/step - accuracy: 0.4984 - loss: 1.1658 - val_accuracy: 0.6220 - val_loss: 1.0439 - learning_rate: 0.0010
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 22s 829ms/step - accuracy: 0.5036 - loss: 1.1311 - val_accuracy: 0.6029 - val_loss: 1.1397 - learning_rate: 0.0010
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 770ms/step - accuracy: 0.5458 - loss: 1.1076

26/26 ━━━━━━━━━━━━━━━━━━━━ 22s 828ms/step - accuracy: 0.5463 - loss: 1.1057 - val_accuracy: 0.6555 - val_loss: 0.9990 - learning_rate: 0.0010
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 846ms/step - accuracy: 0.6037 - loss: 1.1015

26/26 ━━━━━━━━━━━━━━━━━━━━ 24s 917ms/step - accuracy: 0.6032 - loss: 1.1000 - val_accuracy: 0.7416 - val_loss: 0.9588 - learning_rate: 0.0010
Epoch 9/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 814ms/step - accuracy: 0.5848 - loss: 0.9985

26/26 ━━━━━━━━━━━━━━━━━━━━ 23s 865ms/step - accuracy: 0.5849 - loss: 1.0002 - val_accuracy: 0.7368 - val_loss: 0.9365 - learning_rate: 0.0010
Epoch 10/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 22s 863ms/step - accuracy: 0.6277 - loss: 0.9602 - val_accuracy: 0.7273 - val_loss: 0.9695 - learning_rate: 0.0010


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import pickle

# Initialize MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils


def calculate_angle(a, b, c):
    """Calculate the angle between three points"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine_angle, -1, 1)))


def get_angles(landmarks, frame_shape):
    """Extract key joint angles from pose landmarks"""
    points = {}
    indices = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]  # MediaPipe landmarks
    for idx in indices:
        landmark = landmarks[idx]
        points[idx] = (landmark.x * frame_shape[1], landmark.y * frame_shape[0])

    angles = [
        calculate_angle(points[11], points[13], points[15]),  # Left elbow
        calculate_angle(points[12], points[14], points[16]),  # Right elbow
        calculate_angle(points[13], points[11], points[23]),  # Left shoulder
        calculate_angle(points[14], points[12], points[24]),  # Right shoulder
        calculate_angle(points[11], points[23], points[25]),  # Left hip
        calculate_angle(points[12], points[24], points[26]),  # Right hip
        calculate_angle(points[23], points[25], points[27]),  # Left knee
        calculate_angle(points[24], points[26], points[28]),  # Right knee
        calculate_angle(points[11], points[23], points[27])  # Spine
    ]
    return points, np.array(angles)


def get_pose_feedback(pose_name, angles, confidence):
    """Skip detailed checks if confidence is perfect (1.00)"""
    if confidence >= 0.99:  # Using 0.99 to account for floating-point precision
        return ["Perfect form detected! Keep it up!"]

    feedback = []

    # --- Goddess Pose ---
    if pose_name == "goddess":
        feedback.append("GODDESS POSE FEEDBACK:")
        if angles[6] > 120 or angles[7] > 120:
            feedback.append(f"- Bend knees deeper (Current: L{angles[6]:.0f}° R{angles[7]:.0f}°)")
        if angles[8] < 160:
            feedback.append(f"- Flatten lower back (Current: {angles[8]:.0f}°)")

    # --- Warrior II ---
    elif pose_name == "warrior2":
        feedback.append("WARRIOR II FEEDBACK:")
        if angles[6] < 80 or angles[6] > 100:
            feedback.append(f"- Adjust front knee (Current: {angles[6]:.0f}°)")
        if angles[7] < 170:
            feedback.append(f"- Straighten back leg (Current: {angles[7]:.0f}°)")

    # --- Downward Dog ---
    elif pose_name == "downdog":
        feedback.append("DOWNWARD DOG FEEDBACK:")
        if angles[0] > 170 or angles[1] > 170:
            feedback.append("- Microbend elbows")
        if angles[8] < 160:
            feedback.append("- Lift hips higher")

    # --- Tree Pose ---
    elif pose_name == "tree":
        feedback.append("TREE POSE FEEDBACK:")
        if angles[7] < 170:
            feedback.append(f"- Straighten standing leg (Current: {angles[7]:.0f}°)")
        if angles[6] < 70 or angles[6] > 110:
            feedback.append(f"- Adjust raised knee (Current: {angles[6]:.0f}°)")

    return feedback if feedback else ["No significant issues detected"]


def predict_pose(model, img_input, angle_input, idx_to_class):
    """Predict pose using both CNN and angles"""
    pred = model.predict({
        'image_input': np.expand_dims(img_input, axis=0),
        'angle_input': np.expand_dims(angle_input, axis=0)
    }, verbose=0)
    return idx_to_class[np.argmax(pred)], np.max(pred)


def main():
    # Load model and class mapping
    model = tf.keras.models.load_model('best_model.h5')
    with open('class_mapping.pkl', 'rb') as f:
        idx_to_class = pickle.load(f)
        idx_to_class = {int(k): v for k, v in idx_to_class.items()}

    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        # Process frame
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        output = frame.copy()

        if results.pose_landmarks:
            points, angles = get_angles(results.pose_landmarks.landmark, frame.shape)

            # Prepare inputs
            img_input = cv2.resize(image, (224, 224)) / 255.0
            angle_input = angles.astype(np.float32)

            # Predict pose
            pose_name, confidence = predict_pose(model, img_input, angle_input, idx_to_class)

            # Get feedback
            feedback = get_pose_feedback(pose_name, angles, confidence)

            # Display results
            cv2.putText(output, f"Pose: {pose_name} ({confidence:.2f})",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Display angles
            y_pos = 60
            for name, angle in zip(['L-Elbow', 'R-Elbow', 'L-Shoulder', 'R-Shoulder',
                                    'L-Hip', 'R-Hip', 'L-Knee', 'R-Knee', 'Spine'], angles):
                cv2.putText(output, f"{name}: {angle:.1f}°",
                            (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
                y_pos += 20

            # Display feedback (red text)
            y_pos += 20
            for tip in feedback:
                cv2.putText(output, tip, (10, y_pos),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                y_pos += 30

            # Draw landmarks
            mp_drawing.draw_landmarks(
                output, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )
        cv2.imshow('Yoga Pose Coach', output)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()